In [1]:
import arcpy
from arcpy import env
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from pykrige.ok import OrdinaryKriging
from sklearn.metrics import r2_score

In [ ]:
# 读取数据
data = pd.read_csv('../data/test_ph.csv')
data

In [ ]:
# 删除缺失值
data.dropna(inplace=True)

In [ ]:
# 划分测试数据
train_data = data.sample(frac=0.7,random_state=0)   # 70%的数据用于训练
test_data = data.drop(train_data.index)                 # 30%的数据用于测试
train_data.sample()

In [ ]:
# label
label = 'ph'
train_data[label].describe()

In [2]:
# predictor = TabularPredictor(label=label).fit(train_data,time_limit=600)
predictor = TabularPredictor.load("./AutogluonModels/ag-20230731_030007/")

In [ ]:
# 测试数据集
y_test_pred = predictor.predict(test_data.drop(columns=[label]))

In [ ]:
# 训练数据集
y_train_pred = predictor.predict(train_data.drop(columns=[label]))
y_train = train_data[label]
y_test = test_data[label]

In [ ]:
# 计算残差
residuals_test =y_train - y_train_pred
residuals_test

In [ ]:
# # 高斯克里金
# OK = OrdinaryKriging(train_data['X'], train_data['Y'], residuals_test, variogram_model='gaussian')  # 高斯模型
# # 在测试数据上进行预测
# kriging_predictions_test, _ = OK.execute('points', test_data['X'], test_data['Y'])
# predictions_test = y_test + kriging_predictions_test

# 计算R2
r2 = r2_score(y_test, y_test_pred)
r2

In [4]:
# 设置工作环境
env.workspace = "D:\ArcGISProjects\syraster\ph.gdb"

In [8]:
dem_flat = arcpy.RasterToNumPyArray("SY_DEM_fan").flatten()
twi_flat = arcpy.RasterToNumPyArray("SY_TWI_fan").flatten()
tpi_flat = arcpy.RasterToNumPyArray("SY_TPI_fan").flatten()
asp_flat = arcpy.RasterToNumPyArray("SY_ASP_fan").flatten()
slope_flat = arcpy.RasterToNumPyArray("SY_SLOP_fan").flatten()

In [7]:
dem_flat

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
data.describe()

In [9]:
features2 = np.column_stack((asp_flat,slope_flat,dem_flat,twi_flat,tpi_flat))

In [10]:
features2.size

5033420

In [ ]:
test_data.drop(columns=[label])

In [11]:
xunlian_data = pd.DataFrame(features2,columns=['asp','slope','dem','twi','tpi'])
xunlian_data.size

5033420

In [12]:
result_pred = predictor.predict(xunlian_data)

In [15]:
dem_array =arcpy.RasterToNumPyArray("SY_DEM_fan")
dem_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
#
env.extent = "SY_DEM_fan"
result_array = result_pred.values
raster_array = np.reshape(result_array,dem_array.shape)
raster_array

array([[6.509404, 6.509404, 6.509404, ..., 6.509404, 6.509404, 6.509404],
       [6.509404, 6.509404, 6.509404, ..., 6.509404, 6.509404, 6.509404],
       [6.509404, 6.509404, 6.509404, ..., 6.509404, 6.509404, 6.509404],
       ...,
       [6.509404, 6.509404, 6.509404, ..., 6.509404, 6.509404, 6.509404],
       [6.509404, 6.509404, 6.509404, ..., 6.509404, 6.509404, 6.509404],
       [6.509404, 6.509404, 6.509404, ..., 6.509404, 6.509404, 6.509404]],
      dtype=float32)

In [18]:
result_pred.to_csv("./pred.csv")

In [17]:

# 将预测结果输出为栅格数据
out_raster = arcpy.NumPyArrayToRaster(
    raster_array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    5,
    5,
)
out_raster.save("RESULT")
print("完成")


完成


In [19]:
# 获取坐标
dem_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [28]:
dem_array.shape
dem_array[0][0]

0.0

In [26]:

row_index = 0
col_index = 0
cell_size_x = arcpy.Describe("SY_DEM_fan").meanCellWidth
cell_size_y = arcpy.Describe("SY_DEM_fan").meanCellHeight
# x_coord = arcpy.Extent(arcpy.env.extent).XMin + (col_index + 0.5) * cell_size_x
# y_coord = arcpy.Extent(arcpy.env.extent).YMax - (row_index + 0.5) * cell_size_y
x_coord = arcpy.env.extent.XMin + (col_index + 0.5) * cell_size_x
y_coord = arcpy.env.extent.YMax - (row_index + 0.5) * cell_size_y

print("X坐标:", x_coord)
print("Y坐标:", y_coord)

X坐标: 418865.0
Y坐标: 3108760.0
